В данном ноутбуке 
1. Производим оптимизацию гиперпараметров моделей градиентного бустинга. Выводим графики оптимизации
2. Производим сравнение моделей градиентного бустинга со статистическими моделями Auto ARIMA и Экспоненциальное сглаживание

### Установка библиотек и загрузка данных

In [1]:
!pip install optuna
!pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.9/693.9 kB 18.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.4/372.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.9/187.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.6/154.6 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.2/77

In [2]:
!pip install gdown

In [3]:
!gdown --folder https://drive.google.com/drive/folders/1eRYZ2rSuuOR1zpJQE80u8l611g38d7gF?usp=share_link

Retrieving folder list
Processing file 18QDPPbYY_AWPfs33k_IAMDpsxG-Vo7Mf candidates.csv
Processing file 1OC39ZwCwM5wXdbHFT4p8LXrgyNZor7B- clients_all.csv
Retrieving folder 1ATxPNYy1KljycTameZrr9d-mETkulSRi for_predictions
Processing file 1UIhBbvKodLkhJBwLtPON00JKwC_BH_s8 df_retail.csv
Processing file 1IaryMJwlgIkd0Ixd2gq0VF7kPoWlwIFC study_catboost_predictions_1000.pkl
Processing file 1UNmCOPoz7Qy-V49fwKFpcENtid8CxU4T study_lightgbm_predictions_1000.pkl
Processing file 1KBAZSctunXrWFQTikS1oae0X_0IF9FZy study_xgboost_predictions_1000.pkl
Processing file 16JEkw2SylfLyXJ0B3b6cPX6vitFmhPTC lfm_optuna_10epoch.pickle
Processing file 1FHKTNVp965GrGWOyry4CHAUHuKb_jzwB lfm_standart_10epoch.pickle
Processing file 16d_tD15FYMzXZYndc1EzpgRlhtOky2RB products_all.csv
Processing file 1inuQHuUYd3Vka2u2AmbHvaKZWPvt7kfi purchases_all.csv
Processing file 11aD_fDwXZiC2EVsgTbFl4j5rRc-tWDeR study_catboost_100.pkl
Processing file 10Dunvb16noRdNYo1keHD1nE1MTA8AVet study_lightfm_100.pkl
Processing file 10rsC_7

In [4]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
df_retail = pd.read_csv('./data/df_retail.csv')

df_retail['product_id'] = df_retail['product_id'].astype(int)
df_retail['product_id'] = df_retail['product_id'].astype(str)

df_retail['transaction_datetime'] = pd.to_datetime(df_retail['transaction_datetime'])

### 📖 Вспомогательные классы и функции

In [7]:
class time_series_preparation:

    def __init__(self, df_retail):
        """
        Создаём экземпляр класса `time_series_preparation`

        * `df_retail` - датасет с информацией о продажах по одному магазину
        """
        self.df_retail = df_retail
    

    def create_datesets(self, df_retail):
        """
        Формируем таблицы: 
        1. `total_purchases_by_day` - данные об общих продажах каждого товара за каждый день
        2. `total_orders_by_day` - временной ряд количества клиентов
        3. `union_ts` - объединение временных рядов продаж в один датасет
        * `df_retail` - исходный датасет с информацией о продажах по одному магазину
        """
        total_purchases_by_day = df_retail.pivot_table(values='product_quantity', \
                                                    index=[pd.Grouper(key='transaction_datetime', freq='D')],\
                                                    columns='product_id', aggfunc=sum).fillna(0)

        total_orders_by_day = df_retail.pivot_table(values='transaction_id', \
                                                    index=[pd.Grouper(key='transaction_datetime', freq='D')],\
                                                    aggfunc=lambda x: len(x.unique())).fillna(0)

        total_orders_by_day.rename(columns={'transaction_id': 'count_orders'}, inplace=True)

        # создаём объединенённую таблицу TimeSeries
        union_ts = total_purchases_by_day.melt(var_name='product_id', value_name='y', ignore_index=False)
        
        return total_purchases_by_day, total_orders_by_day, union_ts

    def create_features(self, df):
        """
        Создаём фичи для временного ряда на основе временного индекса
        """
        # временные фичи
        df = df.copy()
        df['dayofweek'] = df.index.dayofweek
        # df['quarter'] = df.index.quarter
        df['month'] = df.index.month
        df['year'] = df.index.year
        df['dayofyear'] = df.index.dayofyear
        df['dayofmonth'] = df.index.day
        df['weekofyear'] = df.index.isocalendar().week

        return df

    def create_data_for_single_ts(self, product_id: str):
        """
        Создаём датасет дневных продаж одного товара

        * `product_id` - id товара
        """
        total_purchases_by_day, total_orders_by_day, _ = self.create_datesets(self.df_retail)

        df = total_purchases_by_day[product_id].to_frame(name='y').join(total_orders_by_day)

        df['count_orders_day_before'] = df['count_orders'].shift(1)
        df.loc[df.index[0], 'count_orders_day_before'] = int(round(df['count_orders_day_before'].mean()))
        df['count_orders_day_before'] = df['count_orders_day_before'].astype(int)

        df = df[['y', 'count_orders_day_before']]

        df = self.create_features(df)

        return df
    
    def create_data_for_multiple_ts(self):
        """
        Создаём объединённый датасет из дневных продаж всех товаров
        """

        _, total_orders_by_day, union_ts = self.create_datesets(self.df_retail)

        df = union_ts.join(total_orders_by_day, how='left').sort_values(['product_id', 'transaction_datetime'])

        df['count_orders_day_before'] = df['count_orders'].shift(1)
        df.loc[df.index[0], 'count_orders_day_before'] = int(round(df['count_orders_day_before'].mean()))
        df['count_orders_day_before'] = df['count_orders_day_before'].astype(int)

        df = df[['y', 'product_id', 'count_orders_day_before']]

        df = self.create_features(df)
        df.reset_index(drop=False, inplace=True)

        return df

In [8]:
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler

def train_model_single_ts(model, 
                          df, n=24, 
                          lags=[-1, -2, -12],
                          **params):
    """
    Обучаем авторегрессионную модель для предсказания 
    одного временного ряда
    * `model` - модель для обучения
    * `df` - данные для обучения
    * `n` - размер тестовой выборки, которую предсказываем
    * `lags` - 
    * `params` - параметры модели
    """
    y = TimeSeries.from_series(df['y'])
    features = ['count_orders_day_before', 'dayofweek', 'month', 'dayofmonth', 'weekofyear']
    future_cov = TimeSeries.from_dataframe(df[features])

    y_train, y_val = y[:-n], y[-n:]

    # отобразим наши таргеты в [0;1]
    transformer = Scaler()
    train_transformed = transformer.fit_transform(y_train)
    val_transformed = transformer.transform(y_val)
    series_transformed = transformer.transform(y)

    model = model(lags=lags,
                    lags_future_covariates=[-1], verbose=-1, 
                    **params)
        
    model.fit(series=train_transformed, future_covariates=future_cov, 
              val_series=val_transformed, val_future_covariates=future_cov, verbose=True)
    

    return model, transformer

In [9]:
from darts import TimeSeries

def train_model_multiple_ts(model, 
                            df, n=24,
                            **params):
    """
    Обучаем авторегрессионную модель для предсказания 
    одного временного ряда
    * `model` - модель для обучения
    * `df` - данные для обучения
    * `n` - размер тестовой выборки, которую предсказываем
    * `lags` - 
    * `params` - параметры модели
    """
    def train_split(y, n):
        y_train = [i[:-n] for i in y]
        return y_train

    # convert pandas 
    y_all = TimeSeries.from_group_dataframe(df,
                                            group_cols=['product_id'],
                                            time_col='transaction_datetime',
                                            value_cols=['y'])


    features = ['count_orders_day_before', 'dayofweek', 'month', 'year', 'dayofmonth', 'weekofyear']

    future_cov_all = TimeSeries.from_group_dataframe(df,
                                                    group_cols=['product_id'],
                                                    time_col='transaction_datetime',
                                                    value_cols=features)

    # hold-out last 24 data points
    y_train_all = train_split(y_all, n)
    
    transformer = Scaler()
    y_train_all = transformer.fit_transform(y_train_all)
    
    lags=[-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14]

    model = model(lags=lags,
                  lags_future_covariates=[0],
                  **params)

    model.fit(series=y_train_all, future_covariates=future_cov_all)

    y_pred = transformer.inverse_transform(model.predict(n=n, series=y_train_all, future_covariates=future_cov_all))

    return model, y_pred, y_train_all, y_all, future_cov_all

### Производим оптимизацию

#### LightGBM

In [313]:
import optuna

optuna.logging.set_verbosity(optuna.logging.WARNING)


from darts.models import LightGBMModel
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler

from darts.metrics import (smape, rmse)

import numpy as np

def objective(trial):
    
    def train_split(y, n):
        y_train = [i[:-n] for i in y]
        y_val = [i[-n:] for i in y]
        return y_train, y_val
    
    # оставим тест в 15 дней
    df = ts_prep.create_data_for_multiple_ts()
    df_cut = df[df['transaction_datetime'] < (max(df['transaction_datetime']) - pd.Timedelta(15 , 'd'))]
    
    # определим размер валидации
    n = 15

    y_all = TimeSeries.from_group_dataframe(df_cut,
                                            group_cols=['product_id'],
                                            time_col='transaction_datetime',
                                            value_cols=['y'])


    features = ['count_orders_day_before', 'dayofweek', 'month', 'year', 'dayofmonth', 'weekofyear']

    future_cov_all = TimeSeries.from_group_dataframe(df_cut,
                                                    group_cols=['product_id'],
                                                    time_col='transaction_datetime',
                                                    value_cols=features)

    # поделим на train и validation по n дням
    y_train_all, y_val_all = train_split(y_all, n)
    
    # отскалируем наш ряд, чтобы мы находились в границах [0;1]
    transformer = Scaler()
    train_transformed = transformer.fit_transform(y_train_all)
    val_transformed = transformer.transform(y_val_all)

    params = {'boosting_type': 'gbdt',
             'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
             'max_depth': trial.suggest_int("max_depth", 2, 12),
             'min_child_samples': trial.suggest_int("min_child_samples", 3, 30),
             'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
             'num_leaves': trial.suggest_int("num_leaves", 2, 100),
             'objective': 'regression',
             'random_state': 42,
             'reg_alpha': trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
             'reg_lambda': trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True)}

    lags=[-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14]
    
    model = LightGBMModel(lags=lags,
                          lags_future_covariates=[0],
                          **params)
    
    model.fit(series=train_transformed, future_covariates=future_cov_all, 
              val_series=val_transformed, val_future_covariates=future_cov_all, verbose=False)
    
    y_preds = transformer.inverse_transform(model.predict(series=train_transformed, n=n, future_covariates=future_cov_all))
    rmses = rmse(y_val_all, y_preds, n_jobs=-1)
    rmse_val = np.mean(rmses)
    
    return rmse_val if rmse_val != np.nan else float("inf")

In [ ]:
study = optuna.create_study(direction="minimize")

study.optimize(objective, n_trials=1000, show_progress_bar=True)

print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [315]:
study.best_trial.params

{'learning_rate': 0.011617694239072952,
 'max_depth': 10,
 'min_child_samples': 9,
 'n_estimators': 117,
 'num_leaves': 93,
 'reg_alpha': 3.35381624288517e-05,
 'reg_lambda': 0.00015478040179843215}

In [316]:
import joblib

# сохраняем данные об обучении и лучшие параметры
joblib.dump(study, "./optimized_models/study_lightgbm_predictions_1000.pkl")

['study_lightgbm_predictions_1000.pkl']

#### XGBoost

In [38]:
import optuna

optuna.logging.set_verbosity(optuna.logging.WARNING)


from darts.models import XGBModel
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler

from darts.metrics import (smape, rmse)

import numpy as np

def objective(trial):
    
    def train_split(y, n):
        y_train = [i[:-n] for i in y]
        y_val = [i[-n:] for i in y]
        return y_train, y_val
    
    ts_prep = time_series_preparation(df_retail)
    
    # оставим тест в 15 дней
    df = ts_prep.create_data_for_multiple_ts()
    df_cut = df[df['transaction_datetime'] < (max(df['transaction_datetime']) - pd.Timedelta(15 , 'd'))]
    
    # определим размер валидации
    n = 15

    # convert pandas 
    y_all = TimeSeries.from_group_dataframe(df_cut,
                                            group_cols=['product_id'],
                                            time_col='transaction_datetime',
                                            value_cols=['y'])


    features = ['count_orders_day_before', 'dayofweek', 'month', 'year', 'dayofmonth', 'weekofyear']

    future_cov_all = TimeSeries.from_group_dataframe(df_cut,
                                                    group_cols=['product_id'],
                                                    time_col='transaction_datetime',
                                                    value_cols=features)

    # поделим на train и validation по n дням
    y_train_all, y_val_all = train_split(y_all, n)
    
    # отскалируем наш ряд, чтобы мы находились в границах [0;1]
    transformer = Scaler()
    train_transformed = transformer.fit_transform(y_train_all)
    val_transformed = transformer.transform(y_val_all)

    params = {'booster': 'gbtree',
             'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
             'max_depth': trial.suggest_int("max_depth", 2, 12),
             'min_child_samples': trial.suggest_int("min_child_samples", 3, 30),
              'objective': 'reg:squarederror',
              'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
             'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
             'max_leaves': trial.suggest_int("max_leaves", 2, 100),
             'seed': 42,
             'reg_alpha': trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
             'reg_lambda': trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True)}

    lags=[-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14]
    
    model = XGBModel(lags=lags,
                      lags_future_covariates=[0],
                      **params)
    
    model.fit(series=train_transformed, future_covariates=future_cov_all, 
              val_series=val_transformed, val_future_covariates=future_cov_all, verbose=False)

    # Evaluate how good it is on the validation set
    
    y_preds = transformer.inverse_transform(model.predict(series=train_transformed, n=n, future_covariates=future_cov_all))
    rmses = rmse(y_val_all, y_preds, n_jobs=-1)
    rmse_val = np.mean(rmses)
    
    return rmse_val if rmse_val != np.nan else float("inf")

In [ ]:
study_xgboost = optuna.create_study(direction="minimize")

study_xgboost.optimize(objective, n_trials=1000, show_progress_bar=True)

# Finally, print the best value and best hyperparameters:
print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [329]:
import joblib

joblib.dump(study_xgboost, "./optimized_models/study_xgboost_predictions_1000.pkl")

['study_xgboost_predictions_1000.pkl']

#### CatBoost

In [10]:
import optuna

# изменим уровень детализации логирования, потому что с ним слишком много
# выводится
optuna.logging.set_verbosity(optuna.logging.WARNING)


from darts.models import CatBoostModel
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler

from darts.metrics import (smape, rmse)

import numpy as np

def objective(trial):
    
    def train_split(y, n):
        y_train = [i[:-n] for i in y]
        y_val = [i[-n:] for i in y]
        return y_train, y_val
    
    ts_prep = time_series_preparation(df_retail)
    
    # оставим тест в 15 дней
    df = ts_prep.create_data_for_multiple_ts()
    df_cut = df[df['transaction_datetime'] < (max(df['transaction_datetime']) - pd.Timedelta(15 , 'd'))]
    
    # определим размер валидации
    n = 15

    # convert pandas 
    y_all = TimeSeries.from_group_dataframe(df_cut,
                                            group_cols=['product_id'],
                                            time_col='transaction_datetime',
                                            value_cols=['y'])


    features = ['count_orders_day_before', 'dayofweek', 'month', 'year', 'dayofmonth', 'weekofyear']
    cat_col = ['dayofweek', 'month', 'year', 'dayofmonth', 'weekofyear']

    future_cov_all = TimeSeries.from_group_dataframe(df_cut,
                                                    group_cols=['product_id'],
                                                    time_col='transaction_datetime',
                                                    value_cols=features)

    # поделим на train и validation по n дням
    y_train_all, y_val_all = train_split(y_all, n)
    
    # отскалируем наш ряд, чтобы мы находились в границах [0;1]
    transformer = Scaler()
    train_transformed = transformer.fit_transform(y_train_all)
    val_transformed = transformer.transform(y_val_all)

    params = {'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
              'depth': trial.suggest_int("max_depth", 2, 12),
              'min_child_samples': trial.suggest_int("min_child_samples", 3, 30),
              "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
              "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
              "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
              'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
              'random_state': 42,
              'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 1e-8, 17.0, log=True)}
    
    
    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)
        
    lags=[-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14]
    
    model = CatBoostModel(lags=lags,
                          lags_future_covariates=[0],
                          verbose=0,
                          **params)
    
    model.fit(series=train_transformed, 
              future_covariates=future_cov_all
             )

    # Evaluate how good it is on the validation set
    
    y_preds = transformer.inverse_transform(model.predict(series=train_transformed, n=n, future_covariates=future_cov_all))
    rmses = rmse(y_val_all, y_preds, n_jobs=-1)
    rmse_val = np.mean(rmses)
    
    return rmse_val if rmse_val != np.nan else float("inf")

In [11]:
study_catboost = optuna.create_study(direction="minimize")

study_catboost.optimize(objective, n_trials=1000, show_progress_bar=True)

# Finally, print the best value and best hyperparameters:
print(f"Best value: {study_catboost.best_value}, Best params: {study_catboost.best_trial.params}")

/opt/conda/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/1000 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Best value: 2.0241830377555354, Best params: {'learning_rate': 0.09359485928979411, 'max_depth': 4, 'min_child_samples': 25, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'colsample_bylevel': 0.03318949196042984, 'n_estimators': 121, 'l2_leaf_reg': 15.108655195692583, 'bagging_temperature': 6.0703309289957605}


In [12]:
import joblib

joblib.dump(study_catboost, "./optimized_models/study_catboost_predictions_1000.pkl")

['study_catboost_predictions_1000.pkl']

### Выведем графики оптимизации

In [ ]:
import joblib

# импортируем информацию об оптимизации, чтобы сгенерировать графики
# и получить лучшие параметры
study_lightgbm = joblib.load("./optimized_models/study_lightgbm_predictions_1000.pkl")
study_xgboost = joblib.load("./optimized_models/study_xgboost_predictions_1000.pkl")
study_catboost = joblib.load("./optimized_models/study_catboost_predictions_1000.pkl")

In [ ]:
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances
)

# CATBOOST
fig_catboost = plot_optimization_history(study_catboost)
fig_catboost.show()

In [ ]:
fig_catboost = plot_param_importances(study_catboost)
fig_catboost.show()

In [ ]:
# XGBOOST
fig_xgboost = plot_optimization_history(study_xgboost)
fig_xgboost.show()

In [ ]:
fig_xgboost = plot_param_importances(study_xgboost)
fig_xgboost.show()

In [ ]:
# LIGHTGBM
fig_lightgbm = plot_optimization_history(study_lightgbm)
fig_lightgbm.show()

In [ ]:
fig_lightgbm = plot_param_importances(study_lightgbm)
fig_lightgbm.show()

### Сравнение между собой и статистическими методами

#### 1. Метрики AUTO-ARIMA

In [ ]:
from darts.metrics import (mae, rmse)
from darts.models import AutoARIMA
from darts import TimeSeries
from tqdm import tqdm

t_s_p = time_series_preparation(df_retail)

total_purchases_by_day, total_orders_by_day, union_ts = t_s_p.create_datesets()

maes = []
rmses = []

maes7 = []
rmses7 = []

maes14 = []
rmses14 = []

n = 15

for i in tqdm(total_purchases_by_day.columns):
    y = TimeSeries.from_series(total_purchases_by_day[i])
    
    train, test = y[:-n], y[-n:]
    
    model_aarima = AutoARIMA()
    model_aarima.fit(train)
    prediction_aarima = model_aarima.predict(len(test))
    
    maes.append(mae(prediction_aarima, test))
    rmses.append(rmse(prediction_aarima, test))
    
    mae7 = abs(sum(prediction_aarima[:7]) - sum(test[:7])).values()[0][0]
    rmse7 = (sum(prediction_aarima[:7]) - sum(test[:7])).values()[0][0] ** 2
    maes7.append(mae7)
    rmses7.append(rmse7)
    
    mae14 = abs(sum(prediction_aarima) - sum(test)).values()[0][0]
    rmse14 = (sum(prediction_aarima) - sum(test)).values()[0][0] ** 2
    maes14.append(mae14)
    rmses14.append(rmse14)
    
print('Auto-ARIMA results:')
    
print(f'MAE: {round(sum(maes) / len(maes), 2)}\nRMSE: {round(sum(rmses) / len(rmses), 2)}')

print(f'MAE-7: {round(sum(maes7) / len(maes7), 2)}\nRMSE-7: {round((sum(rmses7) / len(rmses7)) ** 0.5, 2)}')

print(f'MAE-14: {round(sum(maes14) / len(maes14), 2)}\nRMSE-14: {round((sum(rmses14) / len(rmses14)) ** 0.5, 2)}')

#### 2. Метрики Экспоненциального сглаживания

In [ ]:
from darts.metrics import (mae, rmse)
from darts.models import ExponentialSmoothing
from tqdm import tqdm
from darts import TimeSeries

t_s_p = time_series_preparation(df_retail)

total_purchases_by_day, total_orders_by_day, union_ts = t_s_p.create_datesets()

maes = []
rmses = []

maes7 = []
rmses7 = []

maes14 = []
rmses14 = []

n = 15

for i in tqdm(total_purchases_by_day.columns):
    y = TimeSeries.from_series(total_purchases_by_day[i])
    
    train, test = y[:-n], y[-n:]
    
    model_exp_smooth = ExponentialSmoothing()
    model_exp_smooth.fit(train)
    prediction_exp_smooth = model_exp_smooth.predict(len(test))
    
    maes.append(mae(prediction_exp_smooth, test))
    rmses.append(rmse(prediction_exp_smooth, test))
    
    mae7 = abs(sum(prediction_exp_smooth[:7]) - sum(test[:7])).values()[0][0]
    rmse7 = (sum(prediction_exp_smooth[:7]) - sum(test[:7])).values()[0][0] ** 2
    maes7.append(mae7)
    rmses7.append(rmse7)
    
    mae14 = abs(sum(prediction_exp_smooth) - sum(test)).values()[0][0]
    rmse14 = (sum(prediction_exp_smooth) - sum(test)).values()[0][0] ** 2
    maes14.append(mae14)
    rmses14.append(rmse14)
    
print('Exponentional smoothing results:')
    
print(f'MAE: {round(sum(maes) / len(maes), 2)}\nRMSE: {round(sum(rmses) / len(rmses), 2)}')

print(f'MAE-7: {round(sum(maes7) / len(maes7), 2)}\nRMSE-7: {round((sum(rmses7) / len(rmses7)) ** 0.5, 2)}')

print(f'MAE-14: {round(sum(maes14) / len(maes14), 2)}\nRMSE-14: {round((sum(rmses14) / len(rmses14)) ** 0.5, 2)}')

#### 3. Метрики LightGBM

In [ ]:
from darts.metrics import (mae, rmse, mape)
from tqdm import tqdm
from darts import TimeSeries
from darts.models import (LightGBMModel, XGBModel)
import joblib

study_lightgbm = joblib.load("./optimized_models/study_lightgbm_predictions_1000.pkl")

product_ids = set(df_retail['product_id'])

ts_prep = time_series_preparation(df_retail)

maes = []
rmses = []

maes7 = []
rmses7 = []

maes14 = []
rmses14 = []

n = 15

for product_id in tqdm(product_ids):
    
    df = ts_prep.create_data_for_single_ts(product_id)

    # обучаем LightGBM
    model_lgb, y_pred_lgb, y_train_all, y_all, future_cov_all, transf_lgb = train_model_single_ts(LightGBMModel, 
                                                                                                    df, 
                                                                                                    n=15,
                                                                                                    verbose=-1,
                                                                                                    random_state=42,
                                                                                                    **study_lightgbm.best_trial.params
                                                                                                    )
    
    test = y_all[-n:]
    
    maes.append(mae(y_pred_lgb, test))
    rmses.append(rmse(y_pred_lgb, test))
    
    mae7 = abs(sum(y_pred_lgb[:7]) - sum(test[:7])).values()[0][0]
    rmse7 = (sum(y_pred_lgb[:7]) - sum(test[:7])).values()[0][0] ** 2
    maes7.append(mae7)
    rmses7.append(rmse7)
    
    mae14 = abs(sum(y_pred_lgb) - sum(test)).values()[0][0]
    rmse14 = (sum(y_pred_lgb) - sum(test)).values()[0][0] ** 2
    maes14.append(mae14)
    rmses14.append(rmse14)

print('LightGBM results:')
    
print(f'MAE: {round(sum(maes) / len(maes), 2)}\nRMSE: {round(sum(rmses) / len(rmses), 2)}}')

print(f'MAE-7: {round(sum(maes7) / len(maes7), 2)}\nRMSE-7: {round((sum(rmses7) / len(rmses7)) ** 0.5, 2)}')

print(f'MAE-14: {round(sum(maes14) / len(maes14), 2)}\nRMSE-14: {round((sum(rmses14) / len(rmses14)) ** 0.5, 2)}')

#### 4. Метрики XGBoost

In [ ]:
import joblib

study_xgboost = joblib.load("./optimized_models/study_xgboost_predictions_1000.pkl")

ts_prep = time_series_preparation(df_retail)

product_ids = set(df_retail['product_id'])

maes = []
rmses = []

maes7 = []
rmses7 = []

maes14 = []
rmses14 = []

n = 15

for product_id in tqdm(product_ids):
    
    df = ts_prep.create_data_for_single_ts(product_id)
    
    params = {'learning_rate': 0.042917409207049756,
              'max_depth': 5,
              'grow_policy': 'lossguide',
              'n_estimators': 372,
              'max_leaves': 72,
              'reg_alpha': 0.7646635788464918,
              'reg_lambda': 0.005498659844071468}

    # обучаем XGBoost
    model, y_pred, y_train_all, y_all, future_cov_all, transf = train_model_single_ts(XGBModel, 
                                                                                        df, 
                                                                                        n=15,
                                                                                        random_state=42,
                                                                                        **params
                                                                                        )
    
    test = y_all[-n:]
    
    maes.append(mae(y_pred, test))
    rmses.append(rmse(y_pred, test))
    
    mae7 = abs(sum(y_pred[:7]) - sum(test[:7])).values()[0][0]
    rmse7 = (sum(y_pred[:7]) - sum(test[:7])).values()[0][0] ** 2
    maes7.append(mae7)
    rmses7.append(rmse7)
    
    mae14 = abs(sum(y_pred) - sum(test)).values()[0][0]
    rmse14 = (sum(y_pred) - sum(test)).values()[0][0] ** 2
    maes14.append(mae14)
    rmses14.append(rmse14)

print('XGBoost results:')
    
print(f'MAE: {round(sum(maes) / len(maes), 2)}\nRMSE: {round(sum(rmses) / len(rmses), 2)}')

print(f'MAE-7: {round(sum(maes7) / len(maes7), 2)}\nRMSE-7: {round((sum(rmses7) / len(rmses7)) ** 0.5, 2)}')

print(f'MAE-14: {round(sum(maes14) / len(maes14), 2)}\nRMSE-14: {round((sum(rmses14) / len(rmses14)) ** 0.5, 2)}')

#### 5. Метрики CatBoost

In [ ]:
from darts.metrics import (mae, rmse)
from tqdm import tqdm
from darts.models import CatBoostModel

study_catboost = joblib.load("./optimized_models/study_catboost_predictions_1000.pkl")

ts_prep = time_series_preparation(df_retail)

product_ids = set(df_retail['product_id'])

maes = []
rmses = []

maes7 = []
rmses7 = []

maes14 = []
rmses14 = []
j = 0

n = 15

for product_id in tqdm(product_ids):
    
    df = ts_prep.create_data_for_single_ts(product_id)

    # обучаем CatBoost
    try:
        model, y_pred, y_train_all, y_all, future_cov_all, transf = train_model_single_ts(CatBoostModel, 
                                                                                            df, 
                                                                                            n=15,
                                                                                            verbose=-1,
                                                                                            random_state=42,
                                                                                            **study_catboost.best_trial.params
                                                                                            )
    except:
        print(j)
        j += 1
        continue
    
    test = y_all[-n:]
    
    maes.append(mae(y_pred, test))
    rmses.append(rmse(y_pred, test))
    
    mae7 = abs(sum(y_pred[:7]) - sum(test[:7])).values()[0][0]
    rmse7 = (sum(y_pred[:7]) - sum(test[:7])).values()[0][0] ** 2
    maes7.append(mae7)
    rmses7.append(rmse7)
    
    mae14 = abs(sum(y_pred) - sum(test)).values()[0][0]
    rmse14 = (sum(y_pred) - sum(test)).values()[0][0] ** 2
    maes14.append(mae14)
    rmses14.append(rmse14)

print('Catboost results:')
    
print(f'MAE: {round(sum(maes) / len(maes), 2)}\nRMSE: {round(sum(rmses) / len(rmses), 2)}')

print(f'MAE-7: {round(sum(maes7) / len(maes7), 2)}\nRMSE-7: {round((sum(rmses7) / len(rmses7)) ** 0.5, 2)}')

print(f'MAE-14: {round(sum(maes14) / len(maes14), 2)}\nRMSE-14: {round((sum(rmses14) / len(rmses14)) ** 0.5, 2)}')